In [1]:
import pandas as pd
import numpy as np

np.random.seed(79)

%run Python/simulation.py
%run Python/dataNumpy.py

In [2]:
header = ['Mes', 'Dia', 'Hora', 
         'Temperature', 'Wind_Speed', 'Wind_Direction', 'Humidity', 'Air_Density', 'Rain_Status',
         'AreaServico_PMV', 'AreaServico_Temp', 'Circulacao_PMV', 'Circulacao_Temp', 'Casal_PMV', 'Casal_Temp', 'Banheiro_PMV', 'Banheiro_Temp', 'Jardim_PMV', 'Jardim_Temp', 'Solteiro_PMV', 'Solteiro_Temp', 'Sala_PMV', 'Sala_Temp', 'Mezanino_PMV', 'Mezanino_Temp',
         'PortaServicoLeste', 'JanelaServicoSul', 'JanelaCasalNorte', 'PortaCasalOeste', 'JanelaCasalOeste', 'JanelaBanheiroLeste', 'PortaBanheiroOeste', 'JanelaJardimNorte', 'JanelaJardimLeste', 'JanelaSolteiroNorte', 'PortaSolteiroSul', 'JanelaSolteiroSul', 'PortaSalaNorte', 'JanelaSalaNorte', 'PortaSalaLeste', 'JanelaSalaSul', 'JanelaSalaOeste', 'PortaCozinhaLeste', 'JanelaCozinhaSul', 'JanelaMezaninoNorte', 'JanelaMezaninoSul', 'JanelaMezaninoOeste', 'JanelaSegundoSul' 
         ]

print len(header)

48


In [3]:
def changePort(PORT):
    base = open('simulateAndExit_base.m', 'r')
    f = open('simulateAndExit.m', 'w')
    lines = base.readlines()
    lines[37] = str(lines[37]).replace('50001', str(PORT))
    with open('simulateAndExit.m', 'w') as file_handler:
        for item in lines:
            file_handler.write("{}".format(item))
            
    base.close()
    f.close()

In [4]:
def getRandoms():
    lista = np.random.rand(1,23)
    lista = np.around(lista, decimals=2)
    lista[0][0] = 0  # PortaServicoLeste
    lista[0][3] = 0  # PortaCasalOeste
    lista[0][6] = 0  # PortaBanheiroOeste
    lista[0][7] = 1  # JanelaJardimNorte
    lista[0][8] = 1  # JanelaJardimLeste
    lista[0][10] = 0 # PortaSolteiroSul
    lista[0][12] = 0 # PortaSalaNorte
    lista[0][22] = 0 # JanelaSegundoSul
    return lista
    
def getAll1():
    lista = np.ones((1, 23), dtype=np.int)
    lista[0][0] = 0  # PortaServicoLeste
    lista[0][3] = 0  # PortaCasalOeste
    lista[0][6] = 0  # PortaBanheiroOeste
    lista[0][7] = 1  # JanelaJardimNorte
    lista[0][8] = 1  # JanelaJardimLeste
    lista[0][10] = 0 # PortaSolteiroSul
    lista[0][12] = 0 # PortaSalaNorte
    lista[0][22] = 0 # JanelaSegundoSul
    return lista

def getAll0():
    lista = np.zeros((1, 23), dtype=np.int)
    lista[0][0] = 0  # PortaServicoLeste
    lista[0][3] = 0  # PortaCasalOeste
    lista[0][6] = 0  # PortaBanheiroOeste
    lista[0][7] = 1  # JanelaJardimNorte
    lista[0][8] = 1  # JanelaJardimLeste
    lista[0][10] = 0 # PortaSolteiroSul
    lista[0][12] = 0 # PortaSalaNorte
    lista[0][22] = 0 # JanelaSegundoSul
    return lista

In [5]:
testes = ["%03d" % i for i in xrange(17, 151)]

In [ ]:
for ID in testes:
    print ID
    PORT = int("52"+ID)
    changePort(PORT)
    path = "../Data/Canguçu/"
    fileName = "simulation_random_" + ID + ".csv"

    # data = [3]
    # climaticos = [22]
    # aberturas = [23]
    # total = [48]

    NP_data = np.zeros((1, 48))
    NP_salvar = np.zeros((1, 48))

    ## SIMULACAO
    Simulate = Connection(PORT)
    connec = Simulate.start()

    while True:
        data = connec[0].recv(1024)
        #print ("raw data:")
        #print (data)

        if not data:
            break

        #dados direto do matlab em string
        fromMatlab = data.decode("utf-8")

        #lista com
        dataAndTime = fromMatlab.split(",")

        weatherInText = dataAndTime[0].replace("[", "")
        weatherInText = weatherInText.replace("]", "")

        timeInfo = myTime.getTimeInfo(int(dataAndTime[1]))


        #Gera uma lista com os dados climatologicos
        weather = []
        for x in weatherInText.split(" "):
            weather.append(float(x))


        np_timeInfo = np.array([timeInfo])
        np_weather = np.array([weather])



        #openings = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        #np_openings = getAll1()
        #np_openings = getAll0()
        np_openings = getRandoms()
        sendList = np_openings.tolist()[0]



        NP_data = np.column_stack((np_timeInfo, np_weather, np_openings))

        NP_salvar = np.row_stack((NP_salvar, NP_data))

        sendToMatlab = ', '.join(str(float("%.2f" % e)) for e in sendList )

        #print("OUTPUT DATA:")
        #print (sendToMatlab)
        connec[0].sendall(str.encode(sendToMatlab))

    Simulate.close()

    arquivo = pd.DataFrame(NP_salvar)
    arquivo.columns = header
    arquivo = arquivo[2:]
    arquivo = arquivo[:-1]
    arquivo.to_csv(path + fileName, index=False)


    print ("FIM!")


017
PYTHON || simulacao.py: Iniciando simulacao
PYTHON || simulacao.py: Esperando inicializacao do BCVTB
PYTHON || simulation.py: Recebendo dados
PYTHON || simulacao.py: SIMULACAO CONCLUIDA
PYTHON || simulacao.py: TEMPO DA SIMULACAO: 0:07:44.966711
FIM!
018
PYTHON || simulacao.py: Iniciando simulacao
PYTHON || simulacao.py: Esperando inicializacao do BCVTB
PYTHON || simulation.py: Recebendo dados
PYTHON || simulacao.py: SIMULACAO CONCLUIDA
PYTHON || simulacao.py: TEMPO DA SIMULACAO: 0:07:04.148374
FIM!
019
PYTHON || simulacao.py: Iniciando simulacao
PYTHON || simulacao.py: Esperando inicializacao do BCVTB
PYTHON || simulation.py: Recebendo dados
PYTHON || simulacao.py: SIMULACAO CONCLUIDA
PYTHON || simulacao.py: TEMPO DA SIMULACAO: 0:07:04.433689
FIM!
020
PYTHON || simulacao.py: Iniciando simulacao
PYTHON || simulacao.py: Esperando inicializacao do BCVTB
PYTHON || simulation.py: Recebendo dados
PYTHON || simulacao.py: SIMULACAO CONCLUIDA
PYTHON || simulacao.py: TEMPO DA SIMULACAO: 0:07

In [17]:
arquivo['Temperature'].idxmax()

7335

In [18]:
arquivo.groupby(['Mes'])['Temperature'].describe()

,count,mean,std,min,25%,50%,75%,max
Mes,,,,,,,,
1.0,744.0,19.927151,3.693636,12.3,17.200,19.30,22.3,31.6
2.0,672.0,20.969643,3.472168,13.2,18.400,20.65,23.2,30.6
3.0,744.0,19.973118,3.078249,10.4,17.900,19.70,21.9,29.1
4.0,720.0,17.846667,3.913077,7.1,15.300,17.50,20.3,28.4
5.0,744.0,15.301747,4.863478,5.4,11.700,15.50,18.8,27.5
6.0,720.0,10.577778,3.898818,2.6,7.500,10.10,13.0,23.4
7.0,744.0,9.986425,3.582395,1.5,7.400,10.00,12.4,19.5
8.0,744.0,13.947849,5.649616,4.5,9.400,12.10,18.6,28.4
9.0,720.0,12.758750,3.865745,1.7,10.100,12.60,15.3,24.8


In [16]:
arquivo.to_csv(path + fileName, index=False)